Generación del Dataset Variado
Este Jupyter Notebook tiene como objetivo crear un dataset que contenga imágenes variadas de diferentes clases, teniendo en cuenta las columnas 'rev', 'station', 'season' y 'day_part' para garantizar la diversidad de las muestras.

El archivo de entrada es un CSV que contiene información sobre las imágenes, incluyendo la ruta ('path'), la clase ('class'), la revisión ('rev'), la estación ('station'), la cámara ('cam'), la estación del año ('season'), el periodo del día ('day_part') y la fecha y hora ('date_time').

Se excluyen algunas clases específicas, como 'unknown', 'cervid', 'other', 'leporid', 'invertebrate', 'multispecies' y 'small mammal', del dataset final.

El flujo de trabajo es el siguiente:

1. Se lee el archivo CSV utilizando pd.read_csv().
2. Se filtran las filas excluyendo las clases especificadas utilizando ~df['class'].isin(excluded_classes).
3. Se cuenta el número de imágenes por clase.
4. Se define el número máximo de imágenes a tomar por clase.
5. Se crea un DataFrame vacío para almacenar el dataset final.
6. Se itera por cada clase y se realiza lo siguiente:
7. Se obtienen las imágenes de la clase actual.
8. Se determina la cantidad de imágenes a tomar de esta clase.
9. Se agrupa el DataFrame por 'rev', 'station', 'season' y 'day_part'.
10. Se toma una muestra aleatoria de cada grupo utilizando grouped.apply(lambda x: x.sample(n=min(num_images, len(x)), random_state=42)).
11. Se restablece el índice del DataFrame de muestras aleatorias.
12. Se agregan las imágenes al dataset final utilizando pd.concat().
13. Se guarda el dataset final en un nuevo archivo CSV utilizando df_dataset.to_csv().
14. El resultado es un dataset diverso y equilibrado, donde se tomarán hasta 1000 imágenes de cada clase, garantizando la variabilidad en las columnas 'rev', 'station', 'season' y 'day_part'.

Recuerda especificar la ruta de salida adecuada para guardar el archivo CSV generado.

In [18]:
# Librerias
import pandas as pd
import numpy as np

In [19]:
# Constantes

# Ruta de los archivos CSV
csv_path = "../../CSVs/d01_images_data_amplified.csv"

# Definir el número máximo de imágenes a tomar por clase
max_images_per_class = 3000

### Leer el CSVs

In [20]:
# Leer el archivo CSV de entrada
df  = pd.read_csv(csv_path)

In [21]:
# Obtener las clases únicas del DataFrame
classes = df["class"].unique()

In [22]:
# Clases a excluir
excluded_classes = ['unknown', 'cervid', 'other', 'leporid', 'invertebrate', 'multispecies', 'small mammal']

In [23]:
# Filtrar las filas por clases excluidas
df_filtered = df[~df['class'].isin(excluded_classes)]
# Reiniciar los índices del DataFrame df_filtered
df_filtered = df_filtered.reset_index(drop=True)

In [24]:
# Agrupar por clase y contar el número de imágenes
class_counts = df_filtered.groupby('class').size()

In [25]:
# Función para seleccionar las imágenes según los criterios
def select_images(row):
    count = row['count']
    if count <= max_images_per_class:
        selected_images = df_filtered[df_filtered['class'] == row.name]
        return selected_images
    else:
        num_images = min(count, max_images_per_class)
        proportions = {
            'day_part': None,
            'station': None,
            'season': None,
            'rev': None
        }

        # Calcular proporciones automáticamente según las categorías presentes
        for key in proportions:
            if key == 'season':
                season_counts = df_filtered[df_filtered['class'] == row.name]['season'].value_counts()
                seasons = season_counts.index.tolist()
                num_seasons = len(seasons)
                if num_seasons > 0:
                    proportions[key] = {season: 1 / num_seasons for season in seasons}
            elif key == 'day_part':
                day_parts = df_filtered[df_filtered['class'] == row.name]['day_part'].unique()
                num_day_parts = len(day_parts)
                if num_day_parts > 0:
                    proportions[key] = {day_part: 1 / num_day_parts for day_part in day_parts}
            elif key == 'station':
                stations = df_filtered[df_filtered['class'] == row.name]['station'].unique()
                num_stations = len(stations)
                if num_stations > 0:
                    proportions[key] = {station: 1 / num_stations for station in stations}
            elif key == 'rev':
                revs = df_filtered[df_filtered['class'] == row.name]['rev'].unique()
                num_revs = len(revs)
                if num_revs > 0:
                    proportions[key] = {rev: 1 / num_revs for rev in revs}

        selected_images = []
        for key, values in proportions.items():
            if key == 'day_part':
                day_part_counts = df_filtered[df_filtered['class'] == row.name]['day_part'].value_counts()
                for day_part, proportion in values.items():
                    part_images = int(proportion * num_images)
                    if day_part in day_part_counts:
                        part_count = day_part_counts[day_part]
                        if part_count < part_images:
                            selected_images.extend(df_filtered[(df_filtered['class'] == row.name) & (df_filtered['day_part'] == day_part)].index)
                            num_images -= part_count
                        else:
                            sample = df_filtered[(df_filtered['class'] == row.name) & (df_filtered['day_part'] == day_part)].sample(n=part_images).index
                            selected_images.extend(sample)
                            num_images -= part_images
            elif key == 'station':
                station_counts = df_filtered[df_filtered['class'] == row.name]['station'].value_counts()
                for station, proportion in values.items():
                    part_images = int(proportion * num_images)
                    if station in station_counts:
                        part_count = station_counts[station]
                        if part_count < part_images:
                            selected_images.extend(df_filtered[(df_filtered['class'] == row.name) & (df_filtered['station'] == station)].index)
                            num_images -= part_count
                        else:
                            sample = df_filtered[(df_filtered['class'] == row.name) & (df_filtered['station'] == station)].sample(n=part_images).index
                            selected_images.extend(sample)
                            num_images -= part_images
            elif key == 'season':
                season_counts = df_filtered[df_filtered['class'] == row.name]['season'].value_counts()
                for season, proportion in values.items():
                    part_images = int(proportion * num_images)
                    if season in season_counts:
                        part_count = season_counts[season]
                        if part_count < part_images:
                            selected_images.extend(df_filtered[(df_filtered['class'] == row.name) & (df_filtered['season'] == season)].index)
                            num_images -= part_count
                        else:
                            sample = df_filtered[(df_filtered['class'] == row.name) & (df_filtered['season'] == season)].sample(n=part_images).index
                            selected_images.extend(sample)
                            num_images -= part_images
            elif key == 'rev':
                rev_counts = df_filtered[df_filtered['class'] == row.name]['rev'].value_counts()
                for rev, proportion in values.items():
                    part_images = int(proportion * num_images)
                    if rev in rev_counts:
                        part_count = rev_counts[rev]
                        if part_count < part_images:
                            selected_images.extend(df_filtered[(df_filtered['class'] == row.name) & (df_filtered['rev'] == rev)].index)
                            num_images -= part_count
                        else:
                            sample = df_filtered[(df_filtered['class'] == row.name) & (df_filtered['rev'] == rev)].sample(n=part_images).index
                            selected_images.extend(sample)
                            num_images -= part_images

        if num_images > 0:
            remaining_images = df_filtered[df_filtered['class'] == row.name].drop(selected_images).index
            selected_images.extend(remaining_images[:num_images])

        return selected_images


In [26]:
# Aplicar la función de selección de imágenes a cada clase
selected_indices = class_counts.to_frame(name='count').apply(select_images, axis=1).explode().values

In [27]:
selected_df = df_filtered.reindex(selected_indices[8:len(selected_indices)])

In [28]:
# Mostrar el resumen de las imágenes seleccionadas por clase
selected_images_summary = selected_df["class"].value_counts().sort_index()
print("Resumen de imágenes seleccionadas por clase:")
print(selected_images_summary)

Resumen de imágenes seleccionadas por clase:
cow            3000
empty          3000
fallow deer    3000
horse          3000
human          3000
red deer       3000
red fox        3000
wild boar      3000
Name: class, dtype: int64


In [29]:
# Definir la ruta y el nombre del archivo de salida
output_file = "../../CSVs/dataset_ampliado_caltech.csv"
# Guardar el DataFrame filtrado en un nuevo archivo CSV
selected_df.to_csv(output_file, index=False)

### Guardar el DataFrame en un nuevo archivo CSV

In [30]:
df_dataset_output = selected_df[['path', 'class']]

In [31]:
# Definir la ruta y el nombre del archivo de salida
output_file = "../../CSVs/dataset_caltech.csv"

# Guardar el DataFrame filtrado en un nuevo archivo CSV
df_dataset_output.to_csv(output_file, index=False)
